Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3.json")


In [3]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)

    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)
    
  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Searching for the best parameters 🕖

In [4]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v3

In [11]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.05010122259636053,
    'fr_word_p': 0.08218386790030674,
    'learning_rate': 0.005490088979373788,
    'weight_decay': 0.04018550797902253,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 501,
    'epochs': 50,
    'mid_epoch': 50,
    'max_len': 41,
    'end_mark': 3,
    'bleu': 11.4979,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v3_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v3/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 3, evaluation = evaluation, optimizer = Adafactor)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'], config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                # lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### --- Linear

In [10]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/45 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.005434612780854667]}


Test batch number 11: 100%|██████████| 11/11 [00:03<00:00,  3.33batches/s]



Metrics: {'train_loss': 0.23801584173867732, 'test_loss': 0.5316794243725863, 'bleu': 11.8721, 'gen_len': 7.3642}




  2%|▏         | 1/45 [00:36<27:03, 36.90s/it]

...

 98%|█████████▊| 44/45 [18:26<00:24, 24.52s/it]

For epoch 50: 
{Learning rate: [0.0049464222338864]}


Test batch number 11: 100%|██████████| 11/11 [00:03<00:00,  3.54batches/s]



Metrics: {'train_loss': 0.012058688574873003, 'test_loss': 0.6074478544972159, 'bleu': 25.1534, 'gen_len': 7.2407}




100%|██████████| 45/45 [18:51<00:00, 25.13s/it]


### --- Cosine

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### Predictions and Evaluation Wandb v3

In [12]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [13]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 11: 100%|██████████| 11/11 [00:03<00:00,  3.44batches/s]

predictions_: [[0, 55, 9, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 293, 25, 47, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 163, 920, 69, 229, 15, 9, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 108, 271, 104, 155, 56, 20, 77, 26, 25, 12, 4, 1, 0, 0, 0, 0], [0, 34, 26, 153, 148, 225, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 205, 799, 87, 47, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 240, 1422, 87, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 358, 86, 31, 56, 20, 125, 75, 9, 8, 76, 78, 82, 176, 9, 14, 1], [0, 773, 1430, 97, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 246, 67, 903, 180, 92, 466, 26, 673, 3, 6, 1, 0, 0, 0, 0, 0], [0, 644, 602, 8, 125, 334, 12, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 55, 75, 212, 9, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 28, 29, 20, 469, 1416, 26, 155, 396, 4, 1, 0, 0, 0, 0, 0, 0], [0, 28, 29, 20, 59, 12, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 474, 73, 259, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 28, 29, 20, 1334, 26, 92, 72, 118, 3, 262, 102, 4, 1, 0, 0, 0], [0

In [14]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
151,Ceux-ci ne sont peut-être pas partis!,Ñii dañu demul xanaa!,Ñii dañu demul xanaa!
152,En voilà un autre.,Keneen a ŋgi.,Kenn a ŋgi.
153,"Dans ce cas, je pars.","Bon nag, maa ŋgi dem.","Kon nag, dàwul."
154,Où est NGor?,Ana ŋgoor?,Ana kooku.
155,"Je crois que ce n'est pas celui-ci, ce n'est p...","Defe naa du kookuu, du kookee.",Defe naa du kii.
156,C'est leur ami!,Suñu xarit la!,Suñu sarit la!
157,L'enfant dit d'entrer pour qu'il vous voie.,Xale bi ne ŋgeen dugg mu gis leen.,Xale bi bëgg na mayewoon.
158,J'ai vu cet homme qui parlait et qui était ass...,Gis naa góor gi doon wax te toogoon ci suuf.,Gis naa googii gan gi doon wax.
159,Aide-les à cause de ceux-là!,Dimëlé leen ŋgir ñooñe!,Joxal téere bi doomu nit ku yaru kooku!
160,Il demande si tu aurais été.,Mi ŋgiy laaj ndax demkoon ŋga.,Yaa ŋgii demoon.


In [15]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
76,Converser avec lequel?,Waxtaan ak kenn kan?,Waxtaan ak kenn kan?
47,Cet homme qui était parti.,Góor gii demoon.,Góor gii bëggóon.
94,Dites-lui.,Nileen ka.,Nileen leen.
158,J'ai vu cet homme qui parlait et qui était ass...,Gis naa góor gi doon wax te toogoon ci suuf.,Gis naa googii gan gi doon wax.
38,Je l'ai donné à quelqu'un de meilleur que lui.,May naa ka keneen ku ka gën.,Jox naa ka keneen ku sawar.
91,Vers cet autre qui est devant la porte de la m...,Ci keneen ki ci buntu kër gi.,Lépp lu baax la góor gii bëgg.
15,L'homme dit qu'il serait peut-être là.,Góor gi nee na soo demee mi ŋgi fi.,Góor gi née na mi ŋgi fi soo demee.
113,As-tu vu ces autres femmes en question?,Gis ŋga jigéen ñeneen ñooñu?,Gis ŋga jigéen ñeneen ñooñu?
64,C'est celui-là à l'intérieur.,Kookule la ci biir.,"Soo demee, kookule la."
57,Sois homme de ce pays!,Dil nitu réew mi!,Dil nitu réew mi!


In [16]:
df_ft_to_wf[0]

{'test_loss': 0.4957289438356053, 'bleu': 33.2569, 'gen_len': 7.7081}